# Sentiment Analysis mit TF-IDF-Classifier

_Anmerkung: Dieser Artikel ist eine Ergänzung zum its-people-Webinar "Do you speak NLP - Ein Streifzug durch modernes Natural Language Processing mit Python", gehalten im Oktober 2021 von Markus Zeeb und Uwe Blenz (und tatkräftiger Unterstützung durch Andreas Hoffmann und Thomas und Birgit Krämer)._

In diesem Artikel wollen wir uns mit dem Thema "Sentiment Analysis mit TF-IDF Classifiers" beschäftigen. Dazu werden wir uns im ersten Teil allgemein mit TF-IDF befassen, die Grundidee dahinter beleuchten und Schritt für Schritt einen eigenen, kleinen TF-IDF-"Classifier" implementieren (Python). Im zweiten Teil werden wir unsere Erkenntnisse aus dem ersten Teil nutzen, um einen weiteren TF-IDF-Classifier zu bauen und darauf trainieren, Produktreviews von Amazon.com in die Kategorien "positiv" und "negativ" einzuordnen.

## Teil 1 - Kategorisierung von Texten

tf-idf steht für "term frequency - inverse document frequency" und ist ein Verfahren, mit dem Texte (bzw. "Dokumente") in verschiedene Kategorien eingeordnet werden können, bspw. Fachartikel nach Disziplin, Bücher nach Genre usw.

Allgemein gehen wir bei tf-idf davon aus, dass wir eine sehr große Sammlung von Dokumenten (linguist. "Corpus" genannt) haben, die wir klassifizieren wollen; da es uns im Moment aber rein darum geht zu verstehen, wie tf-idf überhaupt funktioniert, nehmen wir für diesen Zweck folgenden, sehr übersichtlichen Corpus an:

In [1]:
documents = [
    "foo bla bla foo",
    "bla bla bar bla",
    "baz bla bla foo bla bla baz",
    "bla bla bla"
]

Unsere Aufgabe soll nun darin bestehen, die Dokumente (wenn möglich), den Kategorien "foo", "bar" und "baz" zuzuordnen. Da unser Beispielskorpus lediglich aus 4 Dokumenten besteht, erkennen wir natürlich sofort, welche Dokumente in welche Kategorien gehören: Wir sehen bspw. sofort, dass der Term "foo" zwei Mal in Dokument 1 enthalten ist und dieses folglich zur Kategorie "foo" gehört. Interessanter wird es in Dokument 3, das zwei Mal "baz" und ein Mal "foo" enthält. Da wir die Zuordnung in mehrere Kategorien erstmal nicht erlauben, nehmen wir für Dokument 3 Kategorie "baz" an, da "baz" häufiger auftritt als "foo".

Wir halten also fest:

> Je häufiger ein Begriff (eng. "term") in einem Dokument vorkommt, desto relevanter (bzw. "informativer") ist er für dieses Dokument.

Diese Beobachtung stellt den Kern des tf-idf-Verfahrens dar und deshalb soll das erste Ziel auf dem Weg zu unserer kleinen tf-idf-Implementierung genau das sein: eine Liste pro Dokument, die angibt, wie oft (häufig) ein Begriff in diesem Dokument vorkommt:

In [2]:
import numpy as np
import pandas as pd

from typing import List, Set

def build_vocabulary(docs: List[str]) -> Set[str]:
    vocabulary = set()
    for doc in docs:
        vocabulary = vocabulary.union(doc.split())
    return vocabulary

def prepare_documents(docs: List[str]) -> pd.DataFrame:
    vocabulary = build_vocabulary(docs)
    results = []
    for doc in docs:
        vec = dict.fromkeys(vocabulary, 0)
        for word in doc.split():
            vec[word] += 1
        results.append(vec)
    return pd.DataFrame(results)

In [3]:
documents = prepare_documents(documents)
documents

,foo,bla,baz,bar
0,2,2,0,0
1,0,3,0,1
2,1,4,2,0
3,0,3,0,0


Wie wir sehen können, sind die Dokumente in unserem Corpus jetzt keine Texte mehr, sondern Listen, wie häufig ein Begriff in einem Dokument vorkommt (inkl. "0 Mal", wenn ein Begriff also nicht im Dokument auftaucht).

Aus technischer Perspektive sind hier zwei Sachen passiert:

1. Die Dokumente wurden vektorisiert, d.h. wir haben textuelle Daten in numerische Vektoren mit einheitlicher Länge überführt.
2. Jeder Vektor ist gleichzeitig auch ein "bag of words", in dem für jeden Term in unserem Corpus angegeben wird, wie oft er in einem bestimmten Dokumten auftaucht.

Theoretisch haben wir damit unser erstes Ziel auch schon erreicht: Wir haben nun für jedes Dokument eine Liste mit "term frequencies", und tatsächlich arbeiten einige Implementierungen von tf-idf mit dieser Definition von "term frequency". 

Wir wollen uns damit aber noch nicht zufrieden geben und definieren "term frequency" als "relative Häufigkeit" von "absolute Häufigkeit eines Terms" zu "Anzahl aller Terme eines Dokuments":

In [4]:
def term_frequency(docs: pd.DataFrame):
    res = []
    for row in docs.values:
        res.append(np.divide(row, row.sum()))
    return pd.DataFrame(res, columns=docs.columns)

term_frequency(documents)

,foo,bla,baz,bar
0,0.500000,0.500000,0.000000,0.00
1,0.000000,0.750000,0.000000,0.25
2,0.142857,0.571429,0.285714,0.00
3,0.000000,1.000000,0.000000,0.00


So weit, so gut. Meinen aufmerksamen Lesern wird aber sicher nicht entgangen sein, dass ich den "Elefanten im Raum" bisher geflissentlich ignoriert habe, nämlich: "bla". 

Der Term "bla" taucht mit Abstand am häufigsten auf und wenn wir uns stumpf an unsere Definition von "Relevanz" weiter oben hielten, gehörten alle unsere Dokumente der Kategorie "bla" an. 

Das möchten wir natürlich unbedingt vermeiden und deshalb den "Relevanz-Score" von Wörtern, die allgemein häufig (d.h. in vielen und unterschiedlichen Dokumenten) in einem Corpus vorkommen, weniger stark gewichtet werden als solche, die allgemein eher selten sind.

Um das zu erreichen, zählen wir erst einmal für jeden Begriff, in wievielen Dokumenten er vorkommt:

In [5]:
def document_frequency(docs: pd.DataFrame):
    return docs.applymap(lambda n: 1 if n > 0 else 0).sum()

document_frequency(documents)

foo    2
bla    4
baz    1
bar    1
dtype: int64

Das führt uns wieder zu einer Häufigkeit, dieses Mal nämlich die sog. "Dokumentenhäufigkeit" (eng. "document frequency"). 

In dieser Form ist document frequency eine absolute Häufigkeit. Würden wir hierzu die relativen Häufigkeiten berechnen (d.h. durch die Gesamtzahl an Dokumenten in unserem Corpus teilen), ergäben sich die folgenden Werte:

In [6]:
document_frequency(documents).div(len(documents))

foo    0.50
bla    1.00
baz    0.25
bar    0.25
dtype: float64

Diese Werte können wir so aber nicht als Gewichte verwenden, da sie immer noch nur solche Begriffe favourisieren, die über den ganzen Corpus hinweg häufig vorkommen. Erreichen wollen wir aber genau das Gegenteil! 

Erinnern wir uns nochmal kurz an den Namen des Verfahrens: "term frequency - **inverse** document frequency" -- "Aha!", wir nun der Lateiner rufen, "_invers_, von lat. _inversus_, also 'umgedreht' oder 'auf den Kopf gestellt'! Könnte das die Lösung sein?" Und tatsächlich, wenn wir die Relation einfach umdrehen und statt document frequency

$$\text{df}(t) = \frac{|\{ doc \in corpus : t \in doc \}|}{N}$$

einfach die "inverse document frequency"

$$\text{idf}(t) = \frac{N}{|\{ doc \in corpus : t \in doc \}|}$$

verwenden (mit $|\{ doc \in corpus : t \in doc \}|$ = "Anz. Dokumente, die _t_ enthalten" und $N$ = "Anz. Dokumente in Corpus"), drehen sich die Gewichtungen tatsächlich um:

$$\text{df}(\text{"bar"}) = \frac{1}{4} < \frac{4}{1} = \text{idf}(\text{"bar"})$$

Oder, weniger mathematisch als Python-Funktion:

In [7]:
def inverse_document_frequency(docs: pd.DataFrame):
    frequencies = document_frequency(docs)
    inverse_frequencies = pd.Series(len(frequencies), index=frequencies.index).div(frequencies)
    return inverse_frequencies.apply(np.log)

inverse_document_frequency(documents)

foo    0.693147
bla    0.000000
baz    1.386294
bar    1.386294
dtype: float64

Wobei wir in unserem Code einen zusätzlichen Schritt eingebaut haben, in dem wir den Logarithmus auf alle Häufigkeiten angewendet haben. Das hat den Grund, dass inverse Häufigkeiten sehr schnell sehr groß werden können (insbes. bei umfangreichen Corpora) und wir diese Entwicklung mittels Logarithmus etwas dämpfen wollen.

Außerdem hat es in unserem Fall den schönen Nebeneffekt, dass Begriffe, die in allen Dokumenten vorkommen (also sehr häufig sind), wegen $log(1) = 0$ quasi gecancelt werden :)

Und damit, liebe Leser, haben wir den Punkt erreicht, an dem wir nur noch die Teile zusammenzufügen brauchen, um den namensgebenden "Star" unseres Artikels zu erhalten:

In [8]:
def term_frequency_inverse_document_frequency(docs: pd.DataFrame) -> pd.DataFrame:
    return term_frequency(docs).mul(inverse_document_frequency(docs))

term_frequency_inverse_document_frequency(documents)

,foo,bla,baz,bar
0,0.346574,0.0,0.000000,0.000000
1,0.000000,0.0,0.000000,0.346574
2,0.099021,0.0,0.396084,0.000000
3,0.000000,0.0,0.000000,0.000000


Das Ergebnis dieser Funktion können wir auch direkt verwenden, um die Dokumente in unserem Corpus zu kategorisieren:

In [9]:
def categorize(docs: pd.DataFrame) -> pd.DataFrame:
    tf_idf = term_frequency_inverse_document_frequency(docs).to_dict('index')
    for doc, record in tf_idf.items():
        max_val = max(record.values())
        key = '-'
        if max_val > 0.0:
            key = [k for k, v in record.items() if v == max_val][0]
        tf_idf[doc]['category'] = key
    return pd.DataFrame.from_dict(tf_idf, orient='index')

categorize(documents)

,foo,bla,baz,bar,category
0,0.346574,0.0,0.000000,0.000000,foo
1,0.000000,0.0,0.000000,0.346574,bar
2,0.099021,0.0,0.396084,0.000000,baz
3,0.000000,0.0,0.000000,0.000000,-


Unsere $\texttt{categorize}()$-Funktion sieht zwar etwas wild aus, das Ergebnis kann sich aber sehen lassen: Für jedes Dokument haben wir den Term mit dem höchsten tf-idf-Score ermittelt und diesen als Kategorienlabel für das Dokument gewählt (bzw. "-" falls kein tf-idf-Score größer 0 ist).

Übrigens haben wir die Labels der Dokumente streng genommen nicht vorgegeben, sondern unsere Funktion sie auswählen lassen. Tf-idf eignet sich daher auch hervorragend zur Exploration von unbekannten Corpora um zu sehen, in welche "natürlichen" Kategorien die Dokumente darin fallen!

Das ist, wie gesagt, eine ungemein nützliche Eigenschaft des tf-idf-Verfahrens; in den meisten Fällen ist es aber umgekehrt und die Kategorien meistens grob schon vorgegeben. In dem Fall könnte eine Kategorisierung so aussehen, dass wir pro Kategorie einen festen Satz "typischer" Begriffe bereits ermittelt haben und unsere categorize-Funktion dann lediglich anhand des häufigsten Begriffes entscheidet, in welche Kategorie ein Dokument fällt. Eine Kategorie "foo" wird vermutlich den Begriff "foo" als typischen Begriff deklarieren, weshalb Dokument 1 (bzw. 0) in unserem Corpus in diese Kategorie fiele.

Ein anderer Ansatz könnte sein, dass wir einen Teil der Dokumente eines Corpus "von Hand" klassifizieren ("labeln") und ein ML-Modell darauf trainieren, die Kategorie eines Dokuments zu erkennen. Diese Idee steckt hinter dem namensgebenden "Sentiment Analysis mit TF-IDF", der wir uns im zweiten Teil zuwenden wollen:

## Teil 2 - Sentiment Analysis mit TF-IDF

Sentiment Analysis ist ein Spezialfall von "document classification", in dem Dokumente (in unserem Fall Produktreviews von Amazon.com) nur zwei Kategorien "positiv" und "negativ" zugeordnet werden. Das Vorgehen hier ist wie folgt:

1. Amazonreviews laden + ein wenig data cleaning + preprocessing
2. Vektorisiere Dokumente aus 1. und berechne tf-idf-Scores
3. Verwende Vektoren aus 2. als Input für Classification

Bis einschließĺich Schritt 2 ist der Ablauf im Grunde derselbe wie im ersten Teil unseres Artikels. Wo wir vorher aber mittels $\texttt{categorize()}$ Labels für unsere Dokumente verteilt haben, haben unsere Dokumente schon Labels, d.h. wir trainieren ein Modell, das auf Grundlage der tf-idf-Scores aus Schritt 2 lernen, welche Dokumente eher positiv und welche eher negativ sind.

Außerdem wollen wir nicht wieder alles "from scratch" implementieren, sondern ungeniert an praxiserprobten Frameworks und Libraries bedienen :)

### Schritt 1: Setting the stage

Wie oben beschrieben, wollen wir uns im ersten Schritt unsere Product-Reviews laden:

In [10]:
amazon_reviews = pd.read_csv('data/amazon_misc_products_reviews.csv', nrows=10_000)
amazon_reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,rating,Time,title,text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Für unsere Zwecke sollen die ersten 10.000 Reviews reichen. Außerdem interessieren uns eigentlich nur die Spalten "rating", "title" und "text" - also weg mit dem Rest!

In [11]:
amazon_reviews = amazon_reviews[['rating', 'title', 'text']]
amazon_reviews.head()

,rating,title,text
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,2,Cough Medicine,If you are looking for the secret ingredient i...
4,5,Great taffy,Great taffy at a great price. There was a wid...


Das sieht schon besser aus, ist aber noch nicht ganz zufriedenstellend. Zum einen sollen die Dokumente nur als "positive" oder "negative" gelabelt sein, d.h. wir wollen die Spalte "rating" von \[1 ... 5\] abbilden auf "0" (_negative_) oder "1" (_positive_). Und zum Zweiten stellen Titel und Inhalt zusammen ein "Dokument" dar, d.h. ich möchte diese beiden Spalten eigentlich gerne in einer einzigen vereinen:

In [12]:
reviews = amazon_reviews[['title', 'text']]
reviews = reviews['title'] + " " + reviews['text']

labels = amazon_reviews[['rating']]
labels = labels['rating'].apply(lambda l: 1 if l > 2 else 0)

reviews.head(), labels.head()

(0    Good Quality Dog Food I have bought several of...
 1    Not as Advertised Product arrived labeled as J...
 2    "Delight" says it all This is a confection tha...
 3    Cough Medicine If you are looking for the secr...
 4    Great taffy Great taffy at a great price.  The...
 dtype: object,
 0    1
 1    0
 2    1
 3    0
 4    1
 Name: rating, dtype: int64)

Ich habe mich dafür entschieden, dass Reviews mit einem Rating zw. 3 und 5 "positiv" sind, ein Rating zw. 1 und 2 dagegen "negativ". Darüber kann man sich nun streiten - besonders auch, ob man für Reviews mit Rating 3 nicht eine Kategorie "neutral" einführen kann. Alles valide Einwände, für unsere Demonstration aber unerheblich :)

Viel wichtiger an dieser Stelle ist der Umstand, dass wir es nicht mehr länger mit einem Mini-Corpus und einem Vokabular aus 4 Fantasiebegriffen zu tun haben, sondern mit tausenden Reviews in natürlicher Sprache (Englisch)!

Der letzte Schritt unseres Preprocessings soll also daraus bestehen, dass wir ein bisschen NLP-Magic über unsere Dokumente träufeln. Dazu holen wir uns ein wenig Hilfe von spacy, einem großartigen NLP-Framework für Python:

In [13]:
import spacy

def lemmatize(doc: str, lemmatizer):
    return ' '.join((t.lemma_.lower() for t in nlp(doc) if t.is_alpha))

def preprocess_documents(docs: pd.Series):
    nlp = spacy.load("en_core_web_sm")
    lemmatizer = nlp.get_pipe("lemmatizer")
    
    return [lemmatize(text, nlp) for text in docs]

Das "heavy lifting" wird übernommen von $\texttt{lemmatize()}$, wo, wer hätte es gedacht, ein Text genommen, von allen "Nicht-Wörtern" (Zahlen, Satz- und Sonderzeichen etc.) befreit und anschließend jedes Wort auf seine Grundform (linguist. "Lemma") reduziert wird.

Hintergrund ist der, dass wir davon ausgehen, dass grammatikalische Aspekte (Einzahl / Mehrzahl, Zeitformen, Aspekt etc.) keine Rolle für das Sentiment eines Dokuments spielt und wir deshalb die Menge an Wörtern in unseren Bags of Words merklich reduzieren können. 

Ein Beispiel dazu: Gramamtikalisch unterscheiden sich die Sätze "The girl is walking the dogs" und "The girl was walking the dog" in Zeit (present, past) und Numerus (dog - dogs), d.h. unser bag of words würde jeweils einen Eintrag für "is", "was" und "dog" und "dogs" enthalten - obwohl diese Unterschiede, wie gesagt, aller Wahrscheinlichkeit nach wenig informativ für das Sentiment eines Dokuments sind.

_Lemmatization_ "entfernt" diese Aspekte:

In [14]:
sentences = [
    "The girl is walking the dogs",
    "The girl was walking the dog"
]

nlp = spacy.load("en_core_web_sm")
lemmatizer = nlp.get_pipe("lemmatizer")

for sentence in sentences:
    print(sentence, "->", lemmatize(sentence, nlp))

The girl is walking the dogs -> the girl be walk the dog
The girl was walking the dog -> the girl be walk the dog


Beide Sätze werden nach dem Preprocessing auf dasselbe Dokument abgebildet - Profit!

### Schritt 2: Vectorization und TF-IDF-Scores

Und damit sind wir auch schon im zweiten Schritt, der "Vektorisierung", angelagt. Theoretisch könnten wir dazu unsere eigene tf-idf-Funktion aus dem ersten Teil verwenden, da der zweite Teil aber unter dem Motto "das Rad nicht neu erfinden" steht, holen wir uns wieder ein wenig Hilfe, dieses Mal nicht bei spacy, sondern dem nicht weniger großartigen (und vermutlich sogar wesentlich verbreiteteren) scikit-learn-Framework:

In [15]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

$\texttt{TfidfVectorizer}$ übernimmt hier den gesamten Prozess der Vektorisierung für uns - wir liefern lediglich die (lemmatisierten) Dokumente als Input. Was der Vectorizer auch macht, ist einige zusätzliche Pre- und Postprocessing-Steps, in unserem Fall bspw. "stopword removal" (= Entfernen von "inhaltsleeren" Begriffen wie Personalpronomen, Konjunktionen etc.), Eindampfen der Bag of Words auf max. 5.000 Begriffe sowie eine gewisse Normalisierung der tf-idf-Scores (L1-Norm, bei Interesse nachzulesen [hier](https://en.wikipedia.org/wiki/Taxicab_geometry)):

In [16]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=5_000, norm='l1')

lemmatized_documents = preprocess_documents(reviews)
features = vectorizer.fit_transform(lemmatized_documents)
features

<10000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 300749 stored elements in Compressed Sparse Row format>

Wie wir sehen können, besteht unsere "Dokumentenmatrix" erwartungsgemäß aus 10.000 Dokumenten ("Reihen") mit 5.000 Termen ("Spalten"). Diese können wir im nächsten Schritt dazu verwenden, ein Classifikation-Model auf unsere Sentiment Analysis-Task zu trainieren. 

### Schritt 3: Train and Predict

Wieder bedienen wir uns dafür scikit-learn:

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

Für unser Modell habe ich mich für eine Support Vector Machine als Classifier entschieden, aus keinem anderen Grund außer dem, dass ich für gewöhnlich bei ML-Tasks immer erst Support Vector Machines ausprobiere (sofern andere Algorithmen nicht offensichtlich besser geeignet sind).

Außerdem habe ich gleich auch Funktionen zum Aufteilen unserer Matrix und Labels in Trainings- und Test-Sets, sowie zum Auswerten der Ergebnisse unserer Klassifizierung.

Als Test-Set werden wir 1.000 gelabelte Dokumente zurückhalten und unser Modell auf 9.000 Dokumenten trainieren:

In [18]:
classifier = SVC()

xtrain, xtest, ytrain, ytest = train_test_split(features, labels, test_size=0.1, random_state=0)
classifier.fit(xtrain, ytrain)
predictions = classifier.predict(xtest)

Schauen wir uns zuerst direkt an, wie hoch die _accuracy_ unseres Modells auf "ungesehenen" Daten ist:

In [19]:
accuracy_score(ytest, predictions)

0.899

89,9% - das ist ziemlich gut! Vielleicht zu gut, denn wie bei fast allen Machine Learning-Projekten besteht die Gefahr, dass wir es hier mit Overfitting zu tun haben.

Werfen wir deshalb ebenfalls einen Blick auf die _confusion matrix_:

In [20]:
pd.DataFrame(confusion_matrix(ytest, predictions), index=['negative (actual)', 'positive (actual)'], columns=['negative (predicted)', 'positive (predicted)'])

,negative (predicted),positive (predicted)
negative (actual),57,100
positive (actual),1,842


Die _confusion matrix_ zeichnet ein leicht anderes Bild: Von den 1.000 Testdokumenten waren 843 positiv, die unser Modell auch bis auf eines korrekt als solche erkannt hat. Von den übrigen 157 Dokumenten, die negativ waren, hat unser Modell aber lediglich 57 korrekt als negativ erkannt, 100 negative Dokumente aber als positiv klassifiziert!

Das heißt, unser Modell ist zwar gut darin, Reviews mit positivem Sentiment als solche zu erkennen, hat aber beim Erkennen von negativen Reviews deutlichen Nachholbedarf!

## Summary

Damit sind wir am Ende unseres kleinen Abenteuers mit TF-IDF und Sentiment Analysis angelangt.

Im ersten Teil haben wir gelernt, dass TF-IDF ein Verfahren ist, mit dem wichtige Begriffe in einem Dokument erkannt werden können und Dokumente anhand dieser Begriffe dann in Kategorien eingeordnet werden können. Daher das auch geschehen kann, ohne dass im Vorfeld feste Kategorien festgelegt wurden, eignet sich tf-idf auch hervorragend zur Exploration von unbekannten Datensätzen.

Sentiment Analysis ist ein Spezialfall von "Zuordnung von Dokumenten zu Kategorien", bei dem die Kategorien bereits feststehen ("positiv" und "negativ") und ein ML-Modell auf vorgelabelten Dokumenten trainiert wird, um später ungelabelte Dokumente diesen beiden Kategorien zuordnen zu können.

Diesen Prozess haben wir im zweiten Teil des Artikels beleuchtet, indem wir eine Support Vector Machine als Classifier trainiert haben, das Produktreviews auf Amazon.com als "positiv" oder "negativ" bewerten kann. Das ging halbwegs gut, wobei das Modell einen deutlichen Bias gegenüber positiven Reviews hatte.

Abschließend hoffe ich, dass ich meinen Lesern das Thema "TF-IDF" näher bringen konnte. Sollten trotzdem noch Fragen offen sein, Sie das Bedürfnis haben, Lob oder Kritk zum Artikel äußern zu wollen oder allgemein Interesse an NLP haben, dann zögern Sie bitte nicht, entweder auf mich direkt (bevorzugt per Mail!) oder auf die Kollegen bei its-people zuzukommen - wir freuen uns :)